# 02 - Simulating One Dimensional Harmonic Oscillators

**Overview** 

This notebook guides you through the simulation of one-dimensional harmonic and quasi-harmonic systems. The notebook will investigate the change in conformation of a conjugated molecule, by looking at the motion around its central dihedral angle. We will use numerical integration of the equation of motion to visualize the motion of the different conformers at low temperature. We will then look at how we can sample the phase space by increasing the temperature or applying an external force. 


In [ ]:
# @title Modules Setup { display-mode: "form" }
import numpy as np
# Install Plotly (if not already)
!pip install -q plotly > /dev/null
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
!pip install -q rdkit > /dev/null
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw

**Problem** 

Let's consider the azobenzene, a simple, photoswitchable organic compound used in functional materials and biomedicine due to its ability to reversibly change shape (cis-trans isomerization) when exposed to different wavelengths of light. This property makes it useful for molecular switches, photoresponsive devices, dyes, and in drug delivery systems, although its toxicity and potential to form benzidine are also important considerations. For this molecule, an important degree of freedom is represented by the rotation around the central double bond. Can we characterize the dynamics of this degree of freedom when we have temperature effects or external forces? 

In [ ]:
# @title Azobenzene Isomers { display-mode: "form" }

# --- Define trans and cis azobenzene using SMILES ---
# Trans-azobenzene (E-isomer)
trans_smiles = "c1ccccc1/N=N/c2ccccc2"

# Cis-azobenzene (Z-isomer)
cis_smiles = "c1ccccc1/N=N\\c2ccccc2"

# Convert to RDKit molecule objects
mol_trans = Chem.MolFromSmiles(trans_smiles)
mol_cis = Chem.MolFromSmiles(cis_smiles)

# Generate 2D coordinates for nice drawing
AllChem.Compute2DCoords(mol_trans)
AllChem.Compute2DCoords(mol_cis)

# Display side by side
Draw.MolsToImage([mol_trans, mol_cis], legends=["Trans-azobenzene", "Cis-azobenzene"])

**Model**

We are going to only consider the dynamics of the dihedral angle as a function of time. We want to characterize the type of dynamics under three different regimes: low temperature, high temperature, under the action of a periodic external force (light).

>Is the motion of the molecule always regular or cahotic? How much do the initial conditions affect our trajectories?

For our simulations to make sense, we need to come up with an analytic function that expresses the energy of the molecule as a function of the dihedral angle. We need this analytical expression to be able to reproduces the main features of the system. We will use a functional form that is commonly adopted in classical force-fields to model dihedral angles:

$$ V^{diehdral}(\phi) = \sum _{k=1}^3 A_k(1 + \cos(k\,\phi+\delta_k)) $$

This energy expression relies on up to three different frequencies in order to modulate peaks and valleys of the potential energy associted with different conformers/isomers. For our cis-trans isomerization we only need the first two terms. Each terms in the above expression requires two empirical parameters $A_k$ (amplitudes) and $\delta_k$ (phases). These parameters can be fitted on quantum-mechanical calculations or they can be adjusted to reproduce experimental results. In order to simplify the problem, we will set all of the $A_k$ parameters to a value of 1.   

## Part 1: Isolated System at Low Temperature


**Questions**

Before you run any of the following cells, answer the following question(s):

1. Think about the energy associated with the change in dihedral angle. Which configuration should be the global minimum in the energy? Are there other local minima? What configurations should have maximum energy? Draw a sketch of the energy's peaks and valleys as the angle $\phi$ is varied between $0$ and $2\pi$, assuming the dihedral angle has a value of 0 for the cis isomer and a value of $\pi$ for the trans isomer. 
2. Consider the energy expression reported above, with $\delta_1 = 0$ and $\delta_2 = \pi$. Evaluate the second derivative of the energy with respect to $\phi$ around the locations of the cis and trans isomers. Are these values the same or different? What do you think these result imply for the dynamics of the molecule at low temperature? 

Divide into small groups and pick one flavor of the numerical integrator. Run the simulation, change the parameters, and run the simulation again as many times as needed to answer the following question(s):

3. Are the results physically sound for any/all of the values of the timestep? What issues, if any, can you see?
4. The animation shows a comparison with an ideal harmonic oscillator (IHO). Under which conditions is the system behaving as an IHO? 
5. How does the trajectory in phase space connect with the type of motion of the system? 
6. How would you decide the best value of the timestep, given the physical properties of the system?
7. The simulation loop is designed to evolve a single degree of freedom, more specifically a dihedral angle. The calculation of the dihedral angle involve considering the positions of four different atoms (a.k.a. it is a four-body interaction term). If you where to consider all the dihedral angles between all the atoms in a molecule, how would the cost of the simulation scale with the system size? How can one reduce this cost? 

**Extra**
Note that some of the questions that applied to the previous notebook are also good reflection points for this experiment:
* What are some of the assumptions/approximations in the model? Some of them are explicitly stated in the paragraph above, some are not explicit. 
* The simulation has a few parameters that you can play with explicitly. Which of these parameters is/are connected to the underlying physics of the problem (physical parameters) and which of these is/are instead connected to the numerical simulation (numerical parameters)?  
* For the physical parameter(s), what experimental observable is connected to the parameters or, alternatively, how would you set their initial values? 

In [ ]:
# @title Energy and Force Definitions and Visualization  { display-mode: "form" }
def potential(phi, A1=1., A2=1.):
    return A2*(1 + np.cos(2*phi+np.pi)) + A1*(1 + np.cos(phi))
def internal_force(phi, A1=1., A2=1.):
    return ( A2 * 2 * np.sin(2*phi+np.pi) + A1 * np.sin(phi) )
def curvature(phi, A1=1., A2=1.):
    return - ( A2 * 4 * np.cos(2*phi+np.pi) + A1 * np.cos(phi) )
def external_force(t, amplitude=0.7, frequency=0.1):
    return amplitude * np.sin(frequency * t)
def damped_force(dphi_dt, gamma=0.1):
    return -gamma * dphi_dt
def total_force(phi, dphi_dt, t, A1=1., A2=1., gamma=0.1, amplitude=0.7, frequency=0.1):
    return internal_force(phi, A1, A2) + external_force(t, amplitude, frequency) + damped_force(dphi_dt, gamma)

def kinetic(dphidt, inertia=1.):
    return 0.5 * inertia * dphidt**2

# --- Grid for plotting ---
phi = np.linspace(0, 2 * np.pi, 500)
v1_of_phi = potential(phi, A1=1., A2=0.)
v2_of_phi = potential(phi, A1=0., A2=1.)
v_of_phi = potential(phi, A1=1., A2=1.)

# --- Plot setup ---
plt.figure(figsize=(8, 5))

plt.plot(phi, v1_of_phi, label='$k$ = 1, Period = 2$\\pi$', linewidth=2, linestyle='--')
plt.plot(phi, v2_of_phi, label='$k$ = 2, Period = $\\pi$', linewidth=2, linestyle='--')
plt.plot(phi, v_of_phi, label='Full Potential', linewidth=2)

# --- Labels and ticks ---
plt.xlabel('Dihedral angle $\\phi$ (rad)', fontsize=12)
plt.ylabel('Potential energy $V(\\phi)$ (a.u.)', fontsize=12)
plt.title('Potential Energy vs Dihedral Angle', fontsize=14)

# --- Plot polish ---
plt.xticks(ticks=[0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi],
           labels=['$0$ = Cis', '$\\frac{\\pi}{2}$', '$\\pi$ = Trans', '$\\frac{3\\pi}{2}$', '$2\\pi$ = Cis'], fontsize=14)
plt.grid(True, linestyle=':', alpha=0.7)
plt.legend(loc='upper center', fontsize=11)
plt.tight_layout()
plt.show()

In [ ]:
# @title Simulation Parameters  { display-mode: "form" }
isomer = "cis"  # @param ["cis", "trans"]
angular_velocity = 0.2  # @param {type:"number"}
inertia = 1 # @param {type:"number"}
dphi_dt = angular_velocity  # initial angular velocity
# simulation parameters
integrator = "Velocity Verlet"  # @param ["Velocity Verlet", "Euler", "RK2", "Leapfrog"]
dt = 0.5  # @param {type:"number"}
nsteps = 100  # @param {type:"integer"}
frame_stride = 1 # @param {type:"integer"}
total_time = nsteps * dt

In [ ]:
# @title Low-T Simulation and Animation  { display-mode: "form" }

# -- Checks ---
if isomer == "cis":
    phi = 0.0  # dihedral angle in radians for cis
else:
    phi = np.pi
# check kinetic energy is less than barrier height
K = kinetic(dphi_dt, inertia)
V_barrier = potential(np.pi/2, A1=1., A2=1.) - potential(phi, A1=1., A2=1.) # this is an approximate estimate
if K >= V_barrier:
    raise ValueError(f"Error: Initial kinetic energy {K:.2f} exceeds barrier height {V_barrier:.2f}. This is a low-T simulation, please reduce angular_velocity.")

# -- Initial conditions ---
phi_not = phi
pot_zero = potential(phi_not, A1=1., A2=1.)  # V(phi0)
kin = kinetic(dphi_dt, inertia)
pot = potential(phi, A1=1., A2=1.) - pot_zero

# -- Time setup ---
time = np.arange(0, total_time, dt)

# --- Analytical Harmonic Solution ---
force_constant = curvature(phi_not, A1=1., A2=1.)  # k = d2V/dphi2 at equilibrium
omega = np.sqrt(force_constant / inertia)  # sqrt(2k/m)
delta_phi = (phi - phi_not) * np.cos(omega * time) + (dphi_dt / omega) * np.sin(omega * time)
phi_harm = phi_not + delta_phi
dphi_dt_harm = - (phi - phi_not) * omega * np.sin(omega * time) + dphi_dt * np.cos(omega * time)
kin_harm = 0.5 * inertia * dphi_dt_harm**2
pot_harm = 0.5 * force_constant * (phi_harm-phi_not)**2
total_harm = kin_harm + pot_harm

trajectory = []
energy = []

# For Leapfrog: initialize half-step velocity
if integrator == "Leapfrog":
    force = internal_force(phi, A1=1., A2=1.)
    dphi_dt -= 0.5 * force / inertia * dt  # backward half-step

# Run the simulation loop
for t in time:
    trajectory.append((phi, dphi_dt))
    energy.append((kin, pot, kin + pot))

    force = internal_force(phi, A1=1., A2=1.)

    if integrator == "Euler":
        # First-order Euler method
        phi += dphi_dt * dt
        dphi_dt += force / inertia * dt

    elif integrator == "Velocity Verlet":
        # Velocity Verlet integrator
        phi += dphi_dt * dt + 0.5 * force / inertia * dt**2
        new_force = internal_force(phi, A1=1., A2=1.)
        dphi_dt += 0.5 * (force + new_force) / inertia * dt

    elif integrator == "RK2":
        # Runge-Kutta 2nd order
        v_half = dphi_dt + 0.5 * dt * force / inertia
        phi += v_half * dt
        force_new = internal_force(phi, A1=1., A2=1.)
        dphi_dt += dt * force_new / inertia  # full step using force at midpoint

    elif integrator == "Leapfrog":
        # Leapfrog method (kick-drift-kick)
        dphi_dt += force / inertia * dt      # full velocity step
        phi += dphi_dt * dt                  # full position step

    kin = kinetic(dphi_dt, inertia)
    pot = potential(phi, A1=1., A2=1.) - pot_zero

# Convert lists to numpy arrays for easier handling
trajectory = np.array(trajectory)
energy = np.array(energy)

# --- Animation with Plotly ---

# --- Create subplot layout ---
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{}, {}],
           [{}, {}],],
    subplot_titles=[
        "Energies vs Time",
        "Position A vs Time",
        "Velocity A vs Time",
        "Phase Space (Position A vs Velocity A)",
    ],
    vertical_spacing=0.1,
    horizontal_spacing=0.15
)
# --- Initial numerical traces (frame 0) ---
fig.add_trace(go.Scatter(x=time, y=kin_harm, mode='lines', line=dict(color='lightgrey', dash='dot'), showlegend=False),
              row=1, col=1)
fig.add_trace(go.Scatter(x=time, y=pot_harm, mode='lines', line=dict(color='lightgrey', dash='dot'), showlegend=False),
              row=1, col=1)
fig.add_trace(go.Scatter(x=time, y=total_harm, mode='lines', line=dict(color='lightgrey', dash='dot'), showlegend=False),
              row=1, col=1)
fig.add_trace(go.Scatter(x=[time[0]], y=[energy[0, 0]], mode='lines',
                         line=dict(color='orange'), name='KE'),
              row=1, col=1)
fig.add_trace(go.Scatter(x=[time[0]], y=[energy[0, 1]], mode='lines',
                         line=dict(color='blue'), name='PE'),
              row=1, col=1)
fig.add_trace(go.Scatter(x=[time[0]], y=[energy[0, 2]], mode='lines',
                         line=dict(color='black'), name='Total E'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=phi_harm, y=time, mode='lines',
                         line=dict(color='lightgrey'), showlegend=False),
              row=1, col=2)
fig.add_trace(go.Scatter(x=[trajectory[0, 0]], y=[time[0]], mode='lines',
                         line=dict(color='blue'), name='xA(t)'),
              row=1, col=2)

fig.add_trace(go.Scatter(x=time, y=dphi_dt_harm, mode='lines',
                         line=dict(color='lightgrey'), showlegend=False),
              row=2, col=1)
fig.add_trace(go.Scatter(x=[time[0]], y=[trajectory[0, 1]], mode='lines',
                         line=dict(color='green'), name='vA(t)'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=phi_harm, y=dphi_dt_harm, mode='markers',
                         line=dict(color='lightgrey'), showlegend=False),
              row=2, col=2)
fig.add_trace(go.Scatter(x=[trajectory[0, 0]], y=[trajectory[0, 1]], mode='markers',
                         line=dict(color='purple'), name='Phase A'),
              row=2, col=2)


# --- Animation frames ---
frames = []
for i in range(0, nsteps, frame_stride):
    frames.append(go.Frame(data=[
        # Row 1, Col 1: Energies vs Time
        go.Scatter(x=time, y=kin_harm, mode='lines', line=dict(color='lightgrey'), showlegend=False),
        go.Scatter(x=time, y=pot_harm, mode='lines', line=dict(color='lightgrey'), showlegend=False),
        go.Scatter(x=time, y=total_harm, mode='lines', line=dict(color='lightgrey'), showlegend=False),
        go.Scatter(x=time[:i], y=energy[:i, 0], mode='lines', line=dict(color='orange')),
        go.Scatter(x=time[:i], y=energy[:i, 1], mode='lines', line=dict(color='blue')),
        go.Scatter(x=time[:i], y=energy[:i, 2], mode='lines', line=dict(color='black')),

        # Row 1, Col 2: Position vs Time (flipped)
        go.Scatter(x=phi_harm, y=time, mode='lines', line=dict(color='lightgrey'), showlegend=False),
        go.Scatter(x=trajectory[:i, 0], y=time[:i], mode='lines', line=dict(color='blue')),

        # Row 2, Col 1: Velocity vs Time
        go.Scatter(x=time, y=dphi_dt_harm, mode='lines', line=dict(color='lightgrey'), showlegend=False),
        go.Scatter(x=time[:i], y=trajectory[:i, 1], mode='lines', line=dict(color='green')),

        # Row 2, Col 2: Phase space
        go.Scatter(x=phi_harm, y=dphi_dt_harm, mode='markers', line=dict(color='lightgrey'), showlegend=False),
        go.Scatter(x=trajectory[:i, 0], y=trajectory[:i, 1], mode='markers', line=dict(color='purple')),

    ]))

# --- Attach frames ---
fig.frames = frames

# --- Layout and buttons ---
total_time = nsteps * dt
fig.update_layout(
    height=1000,
    width=1000,
    title={
    "text": "1D Spring Simulation: Numerical vs Analytical",
    "x": 0.5,             # Centered (try 0.5–0.8 for right alignment)
    "xanchor": "left"     # Align title's anchor point
    },
    showlegend=True,updatemenus=[
    dict(
        type="buttons",
        showactive=True,
        x=0,
        y=1.05,  # instead of 1.15
        xanchor="left",
        yanchor="bottom",
        direction="right",
        buttons=[
            dict(label="▶️ Slow", method="animate",
                 args=[None, {
                     "frame": {"duration": 500, "redraw": True},
                     "fromcurrent": True,
                     "transition": {"duration": 0}
                 }]),
            dict(label="▶️ Medium", method="animate",
                 args=[None, {
                     "frame": {"duration": 100, "redraw": True},
                     "fromcurrent": True,
                     "transition": {"duration": 0}
                 }]),
            dict(label="▶️ Fast", method="animate",
                 args=[None, {
                     "frame": {"duration": 1, "redraw": True},
                     "fromcurrent": True,
                     "transition": {"duration": 0}
                 }]),
            dict(label="⏸ Pause", method="animate",
                 args=[[None], {
                     "mode": "immediate",
                     "frame": {"duration": 0, "redraw": False},
                     "transition": {"duration": 0}
                 }])
        ]
    )
]
)

# Compute shared x_A range
x_min = min(phi_harm.min(), trajectory[:, 0].min())
x_max = max(phi_harm.max(), trajectory[:, 0].max())
fig.update_xaxes(title="$\\phi$", range=[x_min, x_max], row=1, col=2)
fig.update_xaxes(title="$\\phi$", range=[x_min, x_max], row=2, col=2)

# Compute shared v_A range
v_min = min(dphi_dt_harm.min(), trajectory[:, 1].min())
v_max = max(dphi_dt_harm.max(), trajectory[:, 1].max())
fig.update_yaxes(title="$d\\phi/dt$", range=[v_min, v_max], row=2, col=1)
fig.update_yaxes(title="$d\\phi/dt$", range=[v_min, v_max], row=2, col=2)

# --- Axis labels and ranges ---
fig.update_xaxes(title="Time", range=[0, total_time], row=3, col=1)
fig.update_yaxes(title="Energy", range=[energy.min(),energy.max()],row=1, col=1)

fig.update_xaxes(title="$\\phi$", row=1, col=2)
fig.update_yaxes(title="Time", range=[0, total_time], row=1, col=2)

fig.update_xaxes(title="Time", range=[0, total_time], row=2, col=1)
fig.update_yaxes(title="$d\\phi/dt$", row=2, col=1)

fig.update_xaxes(title="$\\phi$", row=2, col=2)
fig.update_yaxes(title="$d\\phi/dt$", row=2, col=2)

# --- Show the animated figure ---
fig.show()

In [ ]:
# @title Compare Harmonic and Actual Frequencies { display-mode: "form" }

# FFT
fft_vals = np.fft.fft(trajectory[:,0])
freqs = np.fft.fftfreq(len(time), d=dt)

# Only look at positive frequencies
positive = freqs > 0
fft_magnitude = np.abs(fft_vals)

# Find index of peak magnitude
peak_index = np.argmax(fft_magnitude[positive])
dominant_freq = freqs[positive][peak_index]

# Dominant frequency from trajectory NOTE: this may not be accurate if the trajectory is too short
print(f"Dominant frequency: {dominant_freq*2*np.pi:.3f} rad/s")
# Theoretical frequency
print(f"Theoretical frequency: {omega:.3f} rad/s")

## Part 2: Forced Oscillations

In this section we will investigate how we can force the system to explore the energy landscape by applying an oscillatory force. This force may be a model for the interaction of the system with an electromagnetic radiation, but mind that the photochemical properties of this molecule are quantum in nature and are connected to electronic excitations. Additionally, we also introduce a disperive term (a damping proportional to the velocity of the motion). The differential equation that we need to solve is now

$$I \, \frac{d^2 \phi(t)}{dt^2} + \gamma \, \frac{d\phi(t)}{dt} = -\frac{dV^{dihedral}(\phi)}{d\phi} + A^{ext}\cos(\omega^{ext}\, t)$$

where two new parameters control the amplitude ($A^{ext}$) and angular frequency ($\omega^{ext}$) of the external force.

**Questions**

Before you run any of the following cells, answer these question(s):

1. What type of trajectory do you expect (periodic, static, chaotic, etc.)? How do you think the parameters of the external force affects the type of trajectory?
2. Is the system still Hamiltonian? Do you think that the energy of the system should be conserved? 

Run the simulation, change the parameters, and run the simulation again as many times as needed to answer the following question(s):

3. Are the results consistent with what you expected? 
4. For what values of the external force parameters is the system able to explore all of the values of the dihedral angle? 
5. How does the trajectory in phase space connect with the type of motion of the system? 
6. How does the trajectory change if you slightly change the starting conditions (angle and angular velocity) of the system?

In [ ]:
# @title Simulation Parameters  { display-mode: "form" }
dihedral = 0 # @param {type:"number"}
phi0 = dihedral%(2*np.pi) # initial dihedral angle in radians
angular_velocity = 0.1 # @param {type:"number"}
dphi_dt0 = angular_velocity  # initial angular velocity
inertia = 1 # @param {type:"number"}
force_amplitude = 0. # @param {type:"number"}
force_frequency = 1.0 # @param {type:"number"}
gamma = 0.00 # @param {type:"number"}
double_trajectory = True # @param {type:"boolean"}
unwrap = True # @param {type:"boolean"}
# simulation parameters
integrator = "Leapfrog"  # @param ["Velocity Verlet", "Euler", "RK2", "Leapfrog"]
dt = 0.01  # @param {type:"number"}
nsteps = 1000  # @param {type:"integer"}
frame_stride = 100 # @param {type:"integer"}
total_time = nsteps * dt

In [ ]:
# @title Forced Oscillations { display-mode: "form" }
phi = phi0
dphi_dt = dphi_dt0

# -- Initial conditions ---
kin = kinetic(dphi_dt, inertia)
pot = potential(phi, A1=1., A2=1.)

# -- Time setup ---
time = np.arange(0, total_time, dt)
trajectory = []
energy = []

# For Leapfrog: initialize half-step velocity
if integrator == "Leapfrog":
    force = total_force(phi, dphi_dt, t=0., A1=1., A2=1., gamma=gamma, amplitude=force_amplitude, frequency=force_frequency)
    dphi_dt -= 0.5 * force / inertia * dt  # backward half-step

# Run the simulation loop
for t in time:
    trajectory.append((phi, dphi_dt))
    energy.append((kin, pot, kin + pot))

    force = total_force(phi, dphi_dt, t, A1=1., A2=1., gamma=gamma, amplitude=force_amplitude, frequency=force_frequency)

    if integrator == "Euler":
        # First-order Euler method
        phi += dphi_dt * dt
        dphi_dt += force / inertia * dt

    elif integrator == "RK2":
        # Runge-Kutta 2nd order
        v_half = dphi_dt + 0.5 * dt * force / inertia
        phi += v_half * dt
        force_new = total_force(phi, v_half, t+0.5*dt, A1=1., A2=1., gamma=gamma, amplitude=force_amplitude, frequency=force_frequency)
        dphi_dt += dt * force_new / inertia  # full step using force at midpoint

    elif integrator == "Leapfrog":
        # Leapfrog method (kick-drift-kick)
        dphi_dt += force / inertia * dt      # full velocity step
        phi += dphi_dt * dt                  # full position step

    phi = phi % (2 * np.pi)  # keep phi within [0, 2pi]
    kin = kinetic(dphi_dt, inertia)
    pot = potential(phi, A1=1., A2=1.)

# Convert lists to numpy arrays for easier handling
trajectory = np.array(trajectory)
energy = np.array(energy)

if double_trajectory:
    phi = phi0 + 0.0001 # slight offset 
    dphi_dt = dphi_dt0
    trajectory_new = []
    # For Leapfrog: initialize half-step velocity
    if integrator == "Leapfrog":
        force = total_force(phi, dphi_dt, t=0., A1=1., A2=1., gamma=gamma, amplitude=force_amplitude, frequency=force_frequency)
        dphi_dt -= 0.5 * force / inertia * dt  # backward half-step

    # Run the simulation loop
    for t in time:
        trajectory_new.append((phi, dphi_dt))
        force = total_force(phi, dphi_dt, t, A1=1., A2=1., gamma=gamma, amplitude=force_amplitude, frequency=force_frequency)
        if integrator == "Euler":
            # First-order Euler method
            phi += dphi_dt * dt
            dphi_dt += force / inertia * dt
        elif integrator == "RK2":
            # Runge-Kutta 2nd order
            v_half = dphi_dt + 0.5 * dt * force / inertia
            phi += v_half * dt
            force_new = total_force(phi, v_half, t+0.5*dt, A1=1., A2=1., gamma=gamma, amplitude=force_amplitude, frequency=force_frequency)
            dphi_dt += dt * force_new / inertia  # full step using force at midpoint
        elif integrator == "Leapfrog":
            # Leapfrog method (kick-drift-kick)
            dphi_dt += force / inertia * dt      # full velocity step
            phi += dphi_dt * dt                  # full position step
        phi = phi % (2 * np.pi)  # keep phi within [0, 2pi]
    trajectory_new = np.array(trajectory_new)


# --- Create subplot layout: 2 rows ---
if not double_trajectory:

    # Extract data
    phi_vals = trajectory[:, 0]
    dphi_dt_vals = trajectory[:, 1]
    kin_vals = energy[:, 0]
    pot_vals = energy[:, 1]
    tot_vals = energy[:, 2]

    # Optionally unwrap phi for smoother phase path (comment this out if not needed)
    if unwrap : phi_vals = np.unwrap(phi_vals)

    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=False,
        row_heights=[0.5, 0.5],
        vertical_spacing=0.1,
        subplot_titles=[
            "Phase Space: $\\phi$ vs $d\\phi/dt}$",
            "Energies Over Time"
        ]
    )

    # --- Initial traces ---
    # Phase space trajectory (row 1)
    fig.add_trace(go.Scatter(
        x=[phi_vals[0]], y=[dphi_dt_vals[0]],
        mode='lines',
        line=dict(color='purple', width=1),
        name='Trajectory',
        showlegend=False
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=[phi_vals[0]], y=[dphi_dt_vals[0]],
        mode='markers',
        marker=dict(size=8, color='red'),
        name='Current point',
        showlegend=False
    ), row=1, col=1)

    # Energy traces (row 2)
    fig.add_trace(go.Scatter(
        x=[time[0]], y=[kin_vals[0]],
        mode='lines',
        line=dict(color='orange'),
        name='Kinetic Energy'
    ), row=2, col=1)

    fig.add_trace(go.Scatter(
        x=[time[0]], y=[pot_vals[0]],
        mode='lines',
        line=dict(color='blue'),
        name='Potential Energy'
    ), row=2, col=1)

    fig.add_trace(go.Scatter(
        x=[time[0]], y=[tot_vals[0]],
        mode='lines',
        line=dict(color='black'),
        name='Total Energy'
    ), row=2, col=1)

    # --- Animation frames ---
    frames = []
    for i in range(1, len(phi_vals), frame_stride):
        frames.append(go.Frame(
            data=[
                # Phase space
                go.Scatter(x=phi_vals[:i], y=dphi_dt_vals[:i], mode='lines', line=dict(color='purple')),
                go.Scatter(x=[phi_vals[i]], y=[dphi_dt_vals[i]], mode='markers', marker=dict(size=8, color='red')),

                # Energies
                go.Scatter(x=time[:i], y=kin_vals[:i], mode='lines', line=dict(color='orange')),
                go.Scatter(x=time[:i], y=pot_vals[:i], mode='lines', line=dict(color='blue')),
                go.Scatter(x=time[:i], y=tot_vals[:i], mode='lines', line=dict(color='black')),
            ]
        ))

    fig.frames = frames

    # --- Layout ---
    fig.update_layout(
        height=800,
        width=800,
        title={
            "text": "Forced Oscillations: Phase Space and Energy",
            "x": 0.5,
            "xanchor": "center"
        },
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            buttons=[dict(label="▶️ Play", method="animate",
                        args=[None, {
                            "frame": {"duration": 50, "redraw": True},
                            "fromcurrent": True,
                            "transition": {"duration": 0}
                        }])]
        )]
    )

    # --- Axis labels ---
    fig.update_xaxes(title="$\\phi$", range=[phi_vals.min(),phi_vals.max()], row=1, col=1)
    fig.update_yaxes(title="$d\\phi/dt$", range=[dphi_dt_vals.min(),dphi_dt_vals.max()], row=1, col=1)
    fig.update_xaxes(title="Time", row=2, col=1, range=[0, total_time])
    fig.update_yaxes(title="Energy", row=2, col=1, range=[energy.min()*1.1, energy.max()*1.1])
else:

    # Extract data
    phi_vals = trajectory[:, 0]
    dphi_dt_vals = trajectory[:, 1]
    phi_vals_new = trajectory_new[:, 0]
    dphi_dt_vals_new = trajectory_new[:, 1]

    # Optionally unwrap phi for smoother phase path (comment this out if not needed)
    if unwrap : 
        phi_vals = np.unwrap(phi_vals)
        phi_vals_new = np.unwrap(phi_vals_new)

    delta_phi = (phi_vals - phi_vals_new)**2

    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=False,
        row_heights=[0.5, 0.5],
        vertical_spacing=0.1,
        subplot_titles=[
            "Phase Space: $\\phi$ vs $d\\phi/dt}$",
            "Trajectory Comparison"
        ]
    )

    # --- Initial traces ---
    # Phase space trajectory (row 1)
    fig.add_trace(go.Scatter(
        x=[phi_vals[0]], y=[dphi_dt_vals[0]],
        mode='lines',
        line=dict(color='purple', width=1),
        name='Trajectory',
        showlegend=False
    ), row=1, col=1)
    fig.add_trace(go.Scatter(
        x=[phi_vals_new[0]], y=[dphi_dt_vals_new[0]],
        mode='lines',
        line=dict(color='green', width=1),
        name='Trajectory',
        showlegend=False
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=[phi_vals[0]], y=[dphi_dt_vals[0]],
        mode='markers',
        marker=dict(size=8, color='red'),
        name='Current point',
        showlegend=False
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=[phi_vals_new[0]], y=[dphi_dt_vals_new[0]],
        mode='markers',
        marker=dict(size=8, color='blue'),
        name='Current point',
        showlegend=False
    ), row=1, col=1)

    # Trajectory difference (row 2)
    fig.add_trace(go.Scatter(
        x=[time[0]], y=[delta_phi[0]],
        mode='lines',
        line=dict(color='orange'),
        name='Squared Difference'
    ), row=2, col=1)

    # --- Animation frames ---
    frames = []
    for i in range(1, len(phi_vals), frame_stride):
        frames.append(go.Frame(
            data=[
                # Phase space
                go.Scatter(x=phi_vals[:i], y=dphi_dt_vals[:i], mode='lines', line=dict(color='purple')),
                go.Scatter(x=[phi_vals[i]], y=[dphi_dt_vals[i]], mode='markers', marker=dict(size=8, color='red')),
                go.Scatter(x=phi_vals_new[:i], y=dphi_dt_vals_new[:i], mode='lines', line=dict(color='green')),
                go.Scatter(x=[phi_vals_new[i]], y=[dphi_dt_vals_new[i]], mode='markers', marker=dict(size=8, color='blue')),

                # Energies
                go.Scatter(x=time[:i], y=delta_phi[:i], mode='lines', line=dict(color='orange')),
            ]
        ))

    fig.frames = frames

    # --- Layout ---
    fig.update_layout(
        height=800,
        width=800,
        title={
            "text": "Forced Oscillations: Phase Space and Energy",
            "x": 0.5,
            "xanchor": "center"
        },
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            buttons=[dict(label="▶️ Play", method="animate",
                        args=[None, {
                            "frame": {"duration": 50, "redraw": True},
                            "fromcurrent": True,
                            "transition": {"duration": 0}
                        }])]
        )]
    )

    # --- Axis labels ---
    fig.update_xaxes(title="$\\phi$", range=[phi_vals.min(),phi_vals.max()], row=1, col=1)
    fig.update_yaxes(title="$d\\phi/dt$", range=[dphi_dt_vals.min(),dphi_dt_vals.max()], row=1, col=1)
    fig.update_xaxes(title="Time", row=2, col=1, range=[0, total_time])
    fig.update_yaxes(title="Trajectory Difference", row=2, col=1, range=[delta_phi.min()*1.1, delta_phi.max()*1.1])

# --- Show the animated figure ---
fig.show()


## Part 3: Langevin Dynamics 

In this section we will investigate what happens when we allow the energy in the dihedral degree of freedom to dissipate in the environment, while at the same time introducing random 'thermal' forces on the system. This approach is meant to model a more realistic dynamic of a system in a thermal environment and the intensity of the random forces is related (by the fluctuation-dissipation theorem) to the physical properties of the system and environment, namely the dissipation drag and the inertia/mass of the degree of freedom. 
$$I \, \frac{d^2 \phi(t)}{dt^2} + \gamma \, \frac{d\phi(t)}{dt} = -\frac{dV^{dihedral}(\phi)}{d\phi} + \eta(t)
$$

where the noise satisfies

$$\langle \eta(t) \rangle = 0, \quad \langle \eta(t)\eta(t') \rangle = 2 \gamma k_B T \, \delta(t - t') $$


**Questions**

Before you run any of the following cells, answer these question(s):

1. Assume we have no random forces, but just energy dissipation. What do you think will happen to the trajectory as you allow energy dissipation?
2. What do you think should happen if we allow thermal energy from the environment to flow into the system? 

As this simulation requires to solve a stochastic differential equation, we are forced to use yet another numerical integrator known as BAOAB. Run the simulation, change the parameters, and run the simulation again as many times as needed to answer the following question(s):

3. Are the results consistent with what you expected? 
4. At what value of the temperature is the system able to explore all of the values of the dihedral angle? 
5. How does the trajectory in phase space connect with the type of motion of the system? 
6. How does the trajectory change if you slightly change the starting conditions (angle and angular velocity) of the system?

In [ ]:
# @title Simulation Parameters  { display-mode: "form" }
dihedral = 3.14 # @param {type:"number"}
phi0 = dihedral%(2*np.pi) # initial dihedral angle in radians
angular_velocity = 0.01  # @param {type:"number"}
dphi_dt0 = angular_velocity  # initial angular velocity
inertia = 1 # @param {type:"number"}
gamma = 0.1 # @param {type:"number"}
temperature = 0.1 # @param {type:"number"}
kbT = temperature # Boltzmann constant times temperature
double_trajectory = False # @param {type:"boolean"}
unwrap = False # @param {type:"boolean"}
# simulation parameters
dt = 0.01  # @param {type:"number"}
nsteps = 10000  # @param {type:"integer"}
frame_stride = 100 # @param {type:"integer"}
total_time = nsteps * dt

In [ ]:
# @title Langevin Simulation and Animation  { display-mode: "form" }
exp_gamma = np.exp(-gamma * dt / inertia)
sigma = np.sqrt((1 - np.exp(-2 * gamma * dt / inertia)) * (inertia * kbT))

time = np.arange(0, total_time, dt)

# -- Initial conditions ---
phi = phi0
dphi_dt = dphi_dt0
trajectory = []
energy = []
np.random.seed(42)  # for reproducibility
# Run the simulation loop
for t in time:
    trajectory.append((phi,dphi_dt))
    energy.append((potential(phi),kinetic(dphi_dt,inertia),potential(phi)+kinetic(dphi_dt,inertia)))

    # B: half velocity step
    dphi_dt += 0.5 * total_force(phi, dphi_dt, t, A1=1., A2=1., gamma=0., amplitude=0.) / inertia * dt

    # A: half position step
    phi += 0.5 * dphi_dt * dt
    phi = phi % (2 * np.pi)

    # O: stochastic Ornstein-Uhlenbeck step
    dphi_dt = exp_gamma * dphi_dt + sigma * np.random.normal()

    # A: half position step
    phi += 0.5 * dphi_dt * dt
    phi = phi % (2 * np.pi)

    # B: half velocity step
    dphi_dt += 0.5 * total_force( phi, dphi_dt, t, A1=1., A2=1., gamma=0., amplitude=0.) / inertia * dt

# Convert lists to numpy arrays for easier handling
trajectory = np.array(trajectory)
energy = np.array(energy)

if double_trajectory:
    # -- Initial conditions ---
    phi = phi0 + 0.0001 # slight offset
    dphi_dt = dphi_dt0
    trajectory_new = []
    np.random.seed(42)  # for reproducibility
    # Run the simulation loop
    for t in time:
        trajectory_new.append((phi,dphi_dt))
        # B: half velocity step
        dphi_dt += 0.5 * total_force( phi, dphi_dt, t, A1=1., A2=1., gamma=0., amplitude=0.) / inertia * dt
        # A: half position step
        phi += 0.5 * dphi_dt * dt
        phi = phi % (2 * np.pi)
        # O: stochastic Ornstein-Uhlenbeck step
        dphi_dt = exp_gamma * dphi_dt + sigma * np.random.normal()
        # A: half position step
        phi += 0.5 * dphi_dt * dt
        phi = phi % (2 * np.pi)
        # B: half velocity step
        dphi_dt += 0.5 * total_force( phi, dphi_dt, t, A1=1., A2=1., gamma=0., amplitude=0. ) / inertia * dt
    # Convert lists to numpy arrays for easier handling
    trajectory_new = np.array(trajectory_new)

if not double_trajectory:
    # Extract phi and dphi/dt
    phi_vals = trajectory[:, 0]
    dphi_dt_vals = trajectory[:, 1]

    window_size = frame_stride  # or set to ~100× your damping timescale
    T_kin_window = np.convolve(dphi_dt_vals**2, np.ones(window_size)/window_size, mode='valid')

    # Optionally unwrap phi for smoother phase path (comment this out if not needed)
    if unwrap: phi_vals = np.unwrap(phi_vals)

    # --- Create subplot layout ---
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=False,
        row_heights=[0.5, 0.5],
        vertical_spacing=0.1,
        subplot_titles=[
            "Phase Space: $\\phi$ vs $d\\phi/dt}$",
            "Kinetic Energy Over Time"
        ]
    )

    # --- Initial traces (frame 0) ---
    # Phase space (row 1)
    fig.add_trace(go.Scatter(x=[phi_vals[0]], y=[dphi_dt_vals[0]],
                            mode='lines', line=dict(color='purple', width=1),
                            name='Trajectory', showlegend=False), row=1, col=1)

    fig.add_trace(go.Scatter(x=[phi_vals[0]], y=[dphi_dt_vals[0]],
                            mode='markers', marker=dict(size=8, color='red'),
                            name='Current point', showlegend=False), row=1, col=1)

    # Kinetic energy (row 2)
    fig.add_trace(go.Scatter(x=[time[0]], y=[T_kin_window[0]],
                            mode='lines', line=dict(color='orange'),
                            name='Kinetic Energy'), row=2, col=1)

    # --- Animation frames ---
    frames = []
    for i in range(1, len(phi_vals), frame_stride):
        frames.append(go.Frame(
            data=[
                # Phase space
                go.Scatter(x=phi_vals[:i], y=dphi_dt_vals[:i],
                        mode='lines', line=dict(color='purple', width=1)),
                go.Scatter(x=[phi_vals[i]], y=[dphi_dt_vals[i]],
                        mode='markers', marker=dict(size=8, color='red')),

                # Kinetic energy
                go.Scatter(x=time[:i], y=T_kin_window[:i],
                        mode='lines', line=dict(color='orange'))
            ]
        ))

    fig.frames = frames

    # --- Layout ---
    fig.update_layout(
        height=800,
        width=800,
        title={
            "text": "Langevin Dynamics: Phase Space and Kinetic Energy",
            "x": 0.5,
            "xanchor": "center"
        },
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            buttons=[dict(label="▶️ Play", method="animate",
                        args=[None, {
                            "frame": {"duration": 50, "redraw": True},
                            "fromcurrent": True,
                            "transition": {"duration": 0}
                        }])]
        )]
    )

    # --- Axes titles ---
    fig.update_xaxes(title="$\\phi$", range=[phi_vals.min(),phi_vals.max()],row=1, col=1)
    fig.update_yaxes(title="$\\dot{\\phi}$", range=[dphi_dt_vals.min(),dphi_dt_vals.max()], row=1, col=1)
    fig.update_xaxes(title="Time", range=[0, total_time], row=2, col=1)
    fig.update_yaxes(title="Kinetic Energy", range=[T_kin_window.min(),T_kin_window.max()],row=2, col=1)

else:
    # Extract phi and dphi/dt
    phi_vals = trajectory[:, 0]
    dphi_dt_vals = trajectory[:, 1]
    phi_vals_new = trajectory_new[:, 0]
    dphi_dt_vals_new = trajectory_new[:, 1]

    # Optionally unwrap phi for smoother phase path (comment this out if not needed)
    if unwrap:
        phi_vals = np.unwrap(phi_vals)
        phi_vals_new = np.unwrap(phi_vals_new)
    # Optionally unwrap phi for smoother phase path (comment this out if not needed)
    if unwrap : 
        phi_vals = np.unwrap(phi_vals)
        phi_vals_new = np.unwrap(phi_vals_new)

    delta_phi = (phi_vals - phi_vals_new)**2

    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=False,
        row_heights=[0.5, 0.5],
        vertical_spacing=0.1,
        subplot_titles=[
            "Phase Space: $\\phi$ vs $d\\phi/dt}$",
            "Trajectory Comparison"
        ]
    )

    # --- Initial traces ---
    # Phase space trajectory (row 1)
    fig.add_trace(go.Scatter(
        x=[phi_vals[0]], y=[dphi_dt_vals[0]],
        mode='lines',
        line=dict(color='purple', width=1),
        name='Trajectory',
        showlegend=False
    ), row=1, col=1)
    fig.add_trace(go.Scatter(
        x=[phi_vals_new[0]], y=[dphi_dt_vals_new[0]],
        mode='lines',
        line=dict(color='green', width=1),
        name='Trajectory',
        showlegend=False
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=[phi_vals[0]], y=[dphi_dt_vals[0]],
        mode='markers',
        marker=dict(size=8, color='red'),
        name='Current point',
        showlegend=False
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=[phi_vals_new[0]], y=[dphi_dt_vals_new[0]],
        mode='markers',
        marker=dict(size=8, color='blue'),
        name='Current point',
        showlegend=False
    ), row=1, col=1)

    # Trajectory difference (row 2)
    fig.add_trace(go.Scatter(
        x=[time[0]], y=[delta_phi[0]],
        mode='lines',
        line=dict(color='orange'),
        name='Squared Difference'
    ), row=2, col=1)

    # --- Animation frames ---
    frames = []
    for i in range(1, len(phi_vals), frame_stride):
        frames.append(go.Frame(
            data=[
                # Phase space
                go.Scatter(x=phi_vals[:i], y=dphi_dt_vals[:i], mode='lines', line=dict(color='purple')),
                go.Scatter(x=[phi_vals[i]], y=[dphi_dt_vals[i]], mode='markers', marker=dict(size=8, color='red')),
                go.Scatter(x=phi_vals_new[:i], y=dphi_dt_vals_new[:i], mode='lines', line=dict(color='green')),
                go.Scatter(x=[phi_vals_new[i]], y=[dphi_dt_vals_new[i]], mode='markers', marker=dict(size=8, color='blue')),

                # Energies
                go.Scatter(x=time[:i], y=delta_phi[:i], mode='lines', line=dict(color='orange')),
            ]
        ))

    fig.frames = frames

    # --- Layout ---
    fig.update_layout(
        height=800,
        width=800,
        title={
            "text": "Langevin Dynamics: Phase Space and Energy",
            "x": 0.5,
            "xanchor": "center"
        },
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            buttons=[dict(label="▶️ Play", method="animate",
                        args=[None, {
                            "frame": {"duration": 50, "redraw": True},
                            "fromcurrent": True,
                            "transition": {"duration": 0}
                        }])]
        )]
    )

    # --- Axis labels ---
    fig.update_xaxes(title="$\\phi$", range=[phi_vals.min(),phi_vals.max()], row=1, col=1)
    fig.update_yaxes(title="$d\\phi/dt$", range=[dphi_dt_vals.min(),dphi_dt_vals.max()], row=1, col=1)
    fig.update_xaxes(title="Time", row=2, col=1, range=[0, total_time])
    fig.update_yaxes(title="Trajectory Difference", row=2, col=1, range=[delta_phi.min()*1.1, delta_phi.max()*1.1])

fig.show()